# Pre-trained model

In [9]:
from tensorflow import keras
import os

base_model = keras.applications.VGG16(
    weights='imagenet',
    input_shape=(224, 224, 3),
    include_top=False)

base_model.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [10]:
base_model.trainable = False

# Add new layers

In [12]:
import pandas as pd

train_map_df = pd.read_csv(os.path.join("data", "train.csv"))
train_map_df['Id'] = train_map_df['Id'].astype('category')
train_map_df.head()

,Image,Id
0,0000e88ab.jpg,w_f48451c
1,0001f9222.jpg,w_c3d896a
2,00029d126.jpg,w_20df2c5
3,00050a15a.jpg,new_whale
4,0005c1ef8.jpg,new_whale


In [14]:
inputs = keras.Input(shape=(224, 224, 3))
x = base_model(inputs, training=False)
x = keras.layers.GlobalAveragePooling2D()(x)
outputs = keras.layers.Dense(len(train_map_df['Id'].unique()))(x) # A Dense classifier with a each class from our training set
model = keras.Model(inputs, outputs)

In [15]:
model.summary()

Model: "functional_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
vgg16 (Functional)           (None, 7, 7, 512)         14714688  
_________________________________________________________________
global_average_pooling2d_1 ( (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 5005)              2567565   
Total params: 17,282,253
Trainable params: 2,567,565
Non-trainable params: 14,714,688
_________________________________________________________________


# Compiling Model

In [16]:
# we have a categorical problem so need to use CategoricalCrossentropy and CategoricalAccuracy
model.compile(loss=keras.losses.CategoricalCrossentropy(from_logits=True), metrics=[keras.metrics.CategoricalAccuracy()])

# Augment the Data

In [17]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
        samplewise_center=True,  # set each sample mean to 0
        rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.1, # Randomly zoom image 
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,
        vertical_flip=False,
        validation_split=0.2)  # 80/20 split train/test

In [20]:
train_it = datagen.flow_from_dataframe(train_map_df,
                                       directory=os.path.join('data', 'train'), 
                                       x_col='Image',
                                       y_col='Id',
                                       target_size=(224, 224), 
                                       color_mode='rgb', 
                                       class_mode='categorical', 
                                       batch_size=8,
                                       subset='training')
valid_it = datagen.flow_from_dataframe(train_map_df,
                                       directory=os.path.join('data', 'train'), 
                                       x_col='Image',
                                       y_col='Id',
                                       target_size=(224, 224), 
                                       color_mode='rgb', 
                                       class_mode='categorical', 
                                       batch_size=8,
                                       subset='validation')

Found 25361 validated image filenames belonging to 5005 classes.
Found 0 validated image filenames belonging to 5005 classes.


In [21]:
model.fit(train_it, steps_per_epoch=12, validation_data=valid_it, validation_steps=4, epochs=20)

Epoch 1/20
12/12 [==============================] - 10s 799ms/step - loss: 8.1892 - categorical_accuracy: 0.2917
Epoch 2/20
12/12 [==============================] - 10s 804ms/step - loss: 7.8991 - categorical_accuracy: 0.3438
Epoch 3/20
12/12 [==============================] - 10s 801ms/step - loss: 8.4433 - categorical_accuracy: 0.3542
Epoch 4/20
12/12 [==============================] - 10s 793ms/step - loss: 8.6435 - categorical_accuracy: 0.3125
Epoch 5/20
12/12 [==============================] - 10s 800ms/step - loss: 9.7473 - categorical_accuracy: 0.3021
Epoch 6/20
12/12 [==============================] - 10s 810ms/step - loss: 9.8760 - categorical_accuracy: 0.2812
Epoch 7/20
12/12 [==============================] - 10s 796ms/step - loss: 10.1645 - categorical_accuracy: 0.2604
Epoch 8/20
12/12 [==============================] - 10s 797ms/step - loss: 9.9697 - categorical_accuracy: 0.3125
Epoch 9/20
12/12 [==============================] - 10s 801ms/step - loss: 8.9798 - categorical